CARGAR LOS DATOS

In [1]:
import pandas as pd

orders  = pd.read_csv(r'data\orders.csv')
order_products_prior = pd.read_csv(r'data\order_products__prior.csv')
order_products_train = pd.read_csv(r'data\order_products__train.csv')
products = pd.read_csv(r'data\products.csv')
aisles = pd.read_csv(r'data\aisles.csv')
user_segmented = pd.read_csv(r'data\segmented_users.csv')

print(orders.columns)
print(order_products_prior.columns)
print(order_products_train.columns)
print(products.columns)
print(user_segmented.columns)
print(aisles.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')
Index(['user_id', 'n_compras', 'last_order_date', 'mean_time_between_orders',
       'Segmento'],
      dtype='object')
Index(['aisle_id', 'aisle'], dtype='object')


HACER UN MERGE DE LOS CSV

In [2]:
order_products = pd.concat([order_products_prior, order_products_train], ignore_index=True)

merged_data = orders.merge(order_products, on="order_id", how="inner")

merged_data = merged_data.merge(products, on="product_id", how="left")

merged_data = merged_data.merge(aisles, on="aisle_id", how="left")

merged_data = merged_data.merge(user_segmented, on="user_id", how="left")

print(merged_data.head())

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN         196                  1          0   
1                     NaN       14084                  2          0   
2                     NaN       12427                  3          0   
3                     NaN       26088                  4          0   
4                     NaN       26405                  5          0   

                              product_name  aisle_id  department_id  \
0                                     S

CREAR DATAFRAMES POR CADA SEGMENTO

In [3]:
df_frecuentes = merged_data.query("Segmento == 'Frecuente'")
df_nuevos = merged_data.query("Segmento == 'Nuevo'")
df_ocasionales = merged_data.query("Segmento == 'Ocasional'")
df_inactivos = merged_data.query("Segmento == 'Inactivo'")

# Mostrar un ejemplo
print(df_frecuentes.shape)
print(df_nuevos.shape)
print(df_ocasionales.shape)
print(df_inactivos.shape)

print(df_frecuentes.columns)


(11091264, 18)
(5091504, 18)
(12656779, 18)
(4979559, 18)
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'aisle', 'n_compras', 'last_order_date',
       'mean_time_between_orders', 'Segmento'],
      dtype='object')


FILTRAR LOS PRODUCTOS MÁS POPULARES DE CADA SEGMENTO

In [4]:
# Contar cuántas veces aparece cada producto
aisle_counts_frecuentes = df_frecuentes['aisle_id'].value_counts()
aisle_counts_nuevos = df_nuevos['aisle_id'].value_counts()
aisle_counts_ocasionales = df_ocasionales['aisle_id'].value_counts()
aisle_counts_inactivos= df_inactivos['aisle_id'].value_counts()

# Filtrar aisles que aparecen en al menos 500 compras (puedes ajustar este número)
popular_aisle_frecuentes = aisle_counts_frecuentes[aisle_counts_frecuentes > 200].index
popular_aisle_nuevos = aisle_counts_nuevos[aisle_counts_nuevos > 200].index
popular_aisle_ocasionales = aisle_counts_ocasionales[aisle_counts_ocasionales > 200].index
popular_aisle_inactivos= aisle_counts_inactivos[aisle_counts_inactivos > 200].index

# Filtrar el dataset para trabajar solo con los productos más comprados
order_aisle_filtered_frecuentes = df_frecuentes[df_frecuentes['aisle_id'].isin(popular_aisle_frecuentes)]
order_aisle_filtered_nuevos = df_nuevos[df_nuevos['aisle_id'].isin(popular_aisle_nuevos)]
order_aisle_filtered_ocasionales = df_ocasionales[df_ocasionales['aisle_id'].isin(popular_aisle_ocasionales)]
order_aisle_filtered_inactivos = df_inactivos[df_inactivos['aisle_id'].isin(popular_aisle_inactivos)]

LIMITAR EL NÚMERO DE TRANSACCIONES

In [5]:
# Tomar una muestra de 500,000 órdenes para reducir memoria
sample_orders_frecuentes = order_aisle_filtered_frecuentes['order_id'].sample(n=150000, random_state=42)
sample_orders_nuevos = order_aisle_filtered_nuevos['order_id'].sample(n=150000, random_state=42)
sample_orders_ocasionales = order_aisle_filtered_ocasionales['order_id'].sample(n=150000, random_state=42)
sample_orders_inactivos = order_aisle_filtered_inactivos['order_id'].sample(n=150000, random_state=42)

# Filtrar las órdenes en la muestra
order_sample_frecuentes = order_aisle_filtered_frecuentes[order_aisle_filtered_frecuentes['order_id'].isin(sample_orders_frecuentes)]
order_sample_nuevos = order_aisle_filtered_nuevos[order_aisle_filtered_nuevos['order_id'].isin(sample_orders_nuevos)]
order_sample_ocasionales = order_aisle_filtered_ocasionales[order_aisle_filtered_ocasionales['order_id'].isin(sample_orders_ocasionales)]
order_sample_inactivos = order_aisle_filtered_inactivos[order_aisle_filtered_inactivos['order_id'].isin(sample_orders_inactivos)]

CREAR LA MATRIZ DE TRANSACCIONES CON One-Hot Encoding

In [6]:
from scipy.sparse import coo_matrix

# Eliminar duplicados
order_sample_filtered_frecuentes = order_sample_frecuentes.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_nuevos = order_sample_nuevos.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_ocasionales = order_sample_ocasionales.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_inactivos = order_sample_inactivos.drop_duplicates(subset=['order_id', 'product_name'])

# Crear la matriz dispersa (sparse matrix)
product_to_id_frecuentes = {product: idx for idx, product in enumerate(order_sample_filtered_frecuentes['product_name'].unique())}
product_to_id_nuevos = {product: idx for idx, product in enumerate(order_sample_filtered_nuevos['product_name'].unique())}
product_to_id_ocasionales = {product: idx for idx, product in enumerate(order_sample_filtered_ocasionales['product_name'].unique())}
product_to_id_inactivos = {product: idx for idx, product in enumerate(order_sample_filtered_inactivos['product_name'].unique())}

order_sample_filtered_frecuentes['product_id_num'] = order_sample_filtered_frecuentes['product_name'].map(product_to_id_frecuentes)
order_sample_filtered_nuevos['product_id_num'] = order_sample_filtered_nuevos['product_name'].map(product_to_id_nuevos)
order_sample_filtered_ocasionales['product_id_num'] = order_sample_filtered_ocasionales['product_name'].map(product_to_id_ocasionales)
order_sample_filtered_inactivos['product_id_num'] = order_sample_filtered_inactivos['product_name'].map(product_to_id_inactivos)

row_frecuentes = order_sample_filtered_frecuentes['order_id'].astype('category').cat.codes
row_nuevos = order_sample_filtered_nuevos['order_id'].astype('category').cat.codes
row_ocasionales = order_sample_filtered_ocasionales['order_id'].astype('category').cat.codes
row_inactivos = order_sample_filtered_inactivos['order_id'].astype('category').cat.codes

col_frecuentes = order_sample_filtered_frecuentes['product_id_num']
col_nuevos = order_sample_filtered_nuevos['product_id_num']
col_ocasionales = order_sample_filtered_ocasionales['product_id_num']
col_inactivos = order_sample_filtered_inactivos['product_id_num']

data_frecuentes = [1] * len(order_sample_filtered_frecuentes)
data_nuevos = [1] * len(order_sample_filtered_nuevos)
data_ocasionales = [1] * len(order_sample_filtered_ocasionales)          
data_inactivos = [1] * len(order_sample_filtered_inactivos)

# Crear la matriz dispersa
basket_sparse_frecuentes = coo_matrix((data_frecuentes, (row_frecuentes, col_frecuentes)), shape=(order_sample_filtered_frecuentes['order_id'].nunique(), len(product_to_id_frecuentes)))
basket_sparse_nuevos = coo_matrix((data_nuevos, (row_nuevos, col_nuevos)), shape=(order_sample_filtered_nuevos['order_id'].nunique(), len(product_to_id_nuevos)))
basket_sparse_ocasionales = coo_matrix((data_ocasionales, (row_ocasionales, col_ocasionales)), shape=(order_sample_filtered_ocasionales['order_id'].nunique(), len(product_to_id_ocasionales)))
basket_sparse_inactivos = coo_matrix((data_inactivos, (row_inactivos, col_inactivos)), shape=(order_sample_filtered_inactivos['order_id'].nunique(), len(product_to_id_inactivos)))

APLICAR EL ALGORITMO APRIORI

In [7]:
from mlxtend.frequent_patterns import apriori, association_rules

basket_df_frecuentes = pd.DataFrame.sparse.from_spmatrix(basket_sparse_frecuentes, columns=product_to_id_frecuentes.keys())
basket_df_nuevos = pd.DataFrame.sparse.from_spmatrix(basket_sparse_nuevos, columns=product_to_id_nuevos.keys())
basket_df_ocasionales = pd.DataFrame.sparse.from_spmatrix(basket_sparse_ocasionales, columns=product_to_id_ocasionales.keys())
basket_df_inactivos = pd.DataFrame.sparse.from_spmatrix(basket_sparse_inactivos, columns=product_to_id_inactivos.keys())

# Convertir los valores de la matriz en binarios (1 si el producto fue comprado, 0 si no)
basket_df_frecuentes = basket_df_frecuentes.applymap(lambda x: 1 if x > 0 else 0)
basket_df_nuevos = basket_df_nuevos.applymap(lambda x: 1 if x > 0 else 0)
basket_df_ocasionales = basket_df_ocasionales.applymap(lambda x: 1 if x > 0 else 0)
basket_df_inactivos = basket_df_inactivos.applymap(lambda x: 1 if x > 0 else 0)

# Aplicar Apriori para obtener los conjuntos frecuentes
frequent_itemsets_frecuentes = apriori(basket_df_frecuentes, min_support=0.01, use_colnames=True)
frequent_itemsets_nuevos = apriori(basket_df_nuevos, min_support=0.01, use_colnames=True)
frequent_itemsets_ocasionales = apriori(basket_df_ocasionales, min_support=0.01, use_colnames=True)
frequent_itemsets_inactivos = apriori(basket_df_inactivos, min_support=0.01, use_colnames=True)

# Generar reglas de asociación
rules_frecuentes = association_rules(frequent_itemsets_frecuentes, metric="lift", min_threshold=1)
rules_nuevos = association_rules(frequent_itemsets_nuevos, metric="lift", min_threshold=1)
rules_ocasionales = association_rules(frequent_itemsets_ocasionales, metric="lift", min_threshold=1)
rules_inactivos = association_rules(frequent_itemsets_inactivos, metric="lift", min_threshold=1)

C:\Users\cesar\AppData\Local\Temp\ipykernel_14824\2499324798.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df_frecuentes = basket_df_frecuentes.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\cesar\AppData\Local\Temp\ipykernel_14824\2499324798.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df_nuevos = basket_df_nuevos.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\cesar\AppData\Local\Temp\ipykernel_14824\2499324798.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df_ocasionales = basket_df_ocasionales.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\cesar\AppData\Local\Temp\ipykernel_14824\2499324798.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df_inactivos = basket_df_inactivos.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\cesar\AppData\Roaming\Python\Python313\site-packages\mlxtend\fre

FILTRAR REGLAS POR LAS MAS CONFIABLES

In [ ]:
confidence = 0.25

rules_sorted_frecuentes = rules_frecuentes.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])
rules_sorted_nuevos = rules_nuevos.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])
rules_sorted_ocasionales = rules_ocasionales.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])
rules_sorted_inactivos = rules_inactivos.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])

rules_filtered_frecuentes = rules_sorted_frecuentes[rules_sorted_frecuentes['confidence'] > confidence]
rules_filtered_nuevos = rules_sorted_nuevos[rules_sorted_nuevos['confidence'] > confidence]
rules_filtered_ocasionales = rules_sorted_ocasionales[rules_sorted_ocasionales['confidence'] > confidence]
rules_filtered_inactivos = rules_sorted_inactivos[rules_sorted_inactivos['confidence'] > confidence]


rules_filtered_frecuentes.to_csv(fr'market_basket_analysis\filtered_rules_{confidence}_frecuentes.csv',index=False)
rules_filtered_nuevos.to_csv(fr'market_basket_analysis\filtered_rules_{confidence}_nuevos.csv',index=False)
rules_filtered_ocasionales.to_csv(fr'market_basket_analysis\filtered_rules_{confidence}_ocasionales.csv',index=False)
rules_filtered_inactivos.to_csv(fr'market_basket_analysis\filtered_rules_{confidence}_inactivos.csv',index=False)

rules_sorted_frecuentes.to_csv(r'market_basket_analysis\sorted_rules_frecuentes.csv',index=False)
rules_sorted_nuevos.to_csv(r'market_basket_analysis\sorted_rules_nuevos.csv',index=False)
rules_sorted_ocasionales.to_csv(r'market_basket_analysis\sorted_rules_ocasionales.csv',index=False)
rules_sorted_inactivos.to_csv(r'market_basket_analysis\sorted_rules_inactivos.csv',index=False)